# SkillForge - Resume Skill Extraction & Career AnalysisExtract and analyze skills from your resume using JAAT (Job Analysis at a Textual Level), then:- Match your skills to O*NET occupations- Analyze skill gaps for target occupations- Generate career pathways- View real market data (wages, demand, trends)## Quick Start1. **Run Sections 1-2**: Setup and initialization2. **Run Section 3**: Load O*NET and DOL data3. **Run Section 4**: Upload your resume PDF (or paste text)4. **Run Sections 5-11**: View skill analysis, occupation matching, gap analysis, pathways, and market data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field
import warnings
import requests
import json as json_lib
import io
from pathlib import Path
warnings.filterwarnings('ignore')

# For PDF file upload and text extraction
try:
    from ipywidgets import FileUpload, Output
    from IPython.display import display
    IPYWIDGETS_AVAILABLE = True
except ImportError:
    IPYWIDGETS_AVAILABLE = False
    print("Note: ipywidgets not available. Install with: pip install ipywidgets")

# Install PDF extraction library if needed
try:
    import PyPDF2
    PDF_LIBRARY = 'PyPDF2'
except ImportError:
    try:
        import pdfplumber
        PDF_LIBRARY = 'pdfplumber'
    except ImportError:
        print("Installing PDF extraction library...")
        try:
            import subprocess
            import sys
            subprocess.check_call([sys.executable, "-m", "pip", "install", "PyPDF2", "--quiet"])
            import PyPDF2
            PDF_LIBRARY = 'PyPDF2'
            print("PyPDF2 installed successfully")
        except:
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", "pdfplumber", "--quiet"])
                import pdfplumber
                PDF_LIBRARY = 'pdfplumber'
                print("pdfplumber installed successfully")
            except Exception as e:
                print(f"WARNING: Could not install PDF library: {e}")
                print("You can still paste resume text manually")
                PDF_LIBRARY = None

import subprocess
import sys

# Download required NLTK resources for JAAT
print("Checking NLTK resources...")
try:
    import nltk
    import ssl

    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

    required_resources = ['punkt', 'punkt_tab', 'stopwords']
    downloaded = []

    for resource in required_resources:
        try:
            found = False
            for path_prefix in ['tokenizers/', 'taggers/', '']:
                try:
                    nltk.data.find(f'{path_prefix}{resource}')
                    found = True
                    break
                except LookupError:
                    continue

            if found:
                print(f"{resource} already available")
            else:
                raise LookupError(f"{resource} not found")
        except LookupError:
            try:
                print(f"Downloading {resource}...")
                nltk.download(resource, quiet=True)
                downloaded.append(resource)
                print(f"{resource} downloaded")
            except Exception as e:
                if resource == 'punkt_tab':
                    try:
                        print(f"Trying alternative download for punkt_tab...")
                        nltk.download('punkt_tab', quiet=False)
                    except:
                        print(f"WARNING: Warning: Could not download {resource}")
                else:
                    print(f"WARNING: Warning: Could not download {resource}: {e}")

    if downloaded:
        print(f"Downloaded {len(downloaded)} NLTK resource(s)")
except ImportError:
    print(" NLTK not available - installing...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "nltk", "--quiet"])
        import nltk
        required_resources = ['punkt', 'punkt_tab', 'stopwords']
        for resource in required_resources:
            try:
                nltk.download(resource, quiet=True)
                print(f"{resource} downloaded")
            except:
                pass
    except Exception as e:
        print(f"WARNING: Warning: Could not install/download NLTK: {e}")

print("\nInitializing JAAT...")
try:
    from jaat.title_matcher import TitleMatch
    from jaat.task_matcher import TaskMatch
    try:
        from jaat.skill_matcher import SkillMatch
    except ImportError:
        SkillMatch = None
    print("JAAT library found (jaat package)")
except ImportError:
    try:
        from JAAT import JAAT
        TitleMatch = JAAT.TitleMatch
        TaskMatch = JAAT.TaskMatch
        SkillMatch = getattr(JAAT, 'SkillMatch', None)
        print("JAAT library found (JAAT package)")
    except ImportError:
        print("Installing JAAT library...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "jaat", "--quiet"])
            from jaat.title_matcher import TitleMatch
            from jaat.task_matcher import TaskMatch
            try:
                from jaat.skill_matcher import SkillMatch
            except ImportError:
                SkillMatch = None
            print("JAAT library installed and imported (jaat package)")
        except:
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", "JAAT", "--quiet"])
                from JAAT import JAAT
                TitleMatch = JAAT.TitleMatch
                TaskMatch = JAAT.TaskMatch
                SkillMatch = getattr(JAAT, 'SkillMatch', None)
                print("JAAT library installed and imported (JAAT package)")
            except Exception as e:
                print(f"WARNING: Error installing JAAT: {e}")
                print("Please install manually: pip install jaat")
                raise

try:
    SM = None

    if SkillMatch:
        try:
            try:
                SM = SkillMatch(threshold=0.8, num_workers=0)
                print("SkillMatch initialized with threshold=0.8, num_workers=0")
            except:
                try:
                    SM = SkillMatch(threshold=0.8)
                    print("SkillMatch initialized with threshold=0.8")
                except:
                    try:
                        SM = SkillMatch(num_workers=0)
                        print("SkillMatch initialized with num_workers=0")
                    except:
                        SM = SkillMatch()
                        print("SkillMatch initialized with default parameters")
        except Exception as e:
            print(f"Warning: Could not initialize SkillMatch: {e}")
            SM = None

    TiM = TitleMatch()
    try:
        import inspect
        sig = inspect.signature(TaskMatch.__init__)
        params = sig.parameters.keys()

        init_kwargs = {}
        if 'threshold' in params:
            init_kwargs['threshold'] = 0.85
            print("TaskMatch initialized with threshold=0.85 (per JAAT docs)")

        if 'num_workers' in params:
            init_kwargs['num_workers'] = 0
            print("TaskMatch initialized with num_workers=0 (multiprocessing disabled)")
        elif 'use_multiprocessing' in params:
            init_kwargs['use_multiprocessing'] = False
            print("TaskMatch initialized with use_multiprocessing=False")

        if init_kwargs:
            TM = TaskMatch(**init_kwargs)
        else:
            TM = TaskMatch()
            print("TaskMatch initialized with default parameters")

    except Exception as e:
        try:
            TM = TaskMatch(threshold=0.85, num_workers=0)
            print("TaskMatch initialized with threshold=0.85, num_workers=0")
        except:
            try:
                TM = TaskMatch(threshold=0.85)
                print("TaskMatch initialized with threshold=0.85")
            except:
                try:
                    TM = TaskMatch(num_workers=0)
                    print("TaskMatch initialized with num_workers=0")
                except:
                    TM = TaskMatch()
                    print("TaskMatch initialized with defaults")

    print("JAAT initialized successfully")
except Exception as e:
    print(f"Error initializing JAAT: {e}")
    SM = None
    TiM = None
    TM = None

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


## 1. Data ModelsData structures for SkillForge:- User profiles (created from JAAT-extracted resume data)- Occupations (from O*NET database)- Market data (from DOL Apprenticeship Data API)- Career pathways and learning recommendations

In [ ]:
@dataclass

class UserProfile:
    """
    Represents a user's skill profile created from JAAT-extracted data.

    Attributes:
    user_id: Unique identifier
    skills: Dict mapping skill_id to proficiency level (1-5 scale)
    experience_years: Years of professional experience
    education_level: Highest education level
    jaat_feature_weights: JAAT-extracted feature vectors from TaskMatch
    (e.g., {'textual_skill_python': 0.83, ...})
    current_occupation: Current occupation code (O*NET-SOC)
    """
    user_id: str
    skills: Dict[str, int]  # skill_id -> proficiency_level (1-5)
    experience_years: int
    education_level: str
    jaat_feature_weights: Dict[str, float] = None
    current_occupation: Optional[str] = None

@dataclass

class Skill:
    """Represents a skill requirement."""
    skill_id: str
    skill_name: str
    importance_level: int  # 1-5 scale
    category: str = "general"

@dataclass

class KnowledgeRequirement:
    """Represents a knowledge requirement for an occupation."""
    knowledge_id: str
    knowledge_name: str
    importance_level: int  # 1-5 scale

@dataclass

class Occupation:
    """Represents an O*NET occupation."""
    soc_code: str
    title: str
    description: str
    required_skills: Dict[str, int]  # skill_id -> importance (1-5)
    required_knowledge: Dict[str, int] = field(default_factory=dict)  # knowledge_id -> importance
    education_level: str = "Bachelor's degree"
    tasks: List[str] = field(default_factory=list)
    work_context: Dict[str, str] = field(default_factory=dict)

@dataclass

class MarketData:
    """Market data for an occupation (wages, demand, trends)."""
    occupation_code: str
    median_salary: Optional[float] = None
    p10_salary: Optional[float] = None
    p90_salary: Optional[float] = None
    starting_wage: Optional[float] = None
    exit_wage: Optional[float] = None
    state: Optional[str] = None
    demand_trend: Optional[str] = None  # "growing", "stable", "declining"
    growth_rate: Optional[float] = None

@dataclass

class JobPosting:
    """Represents a job posting."""
    title: str
    description: str
    required_skills: Dict[str, int] = field(default_factory=dict)
    salary_range: Optional[Tuple[float, float]] = None
    location: Optional[str] = None
    tools_technologies: List[str] = field(default_factory=list)
    matched_occupation_code: Optional[str] = None

@dataclass

class CareerPathway:
    """Represents a career pathway from current to target occupation."""
    current_occupation: Occupation
    target_occupation: Occupation
    steps: List[Dict] = field(default_factory=list)  # List of {occupation, required_skills, estimated_time}
    total_estimated_time: Optional[int] = None  # months
    salary_trajectory: List[Tuple[str, float]] = field(default_factory=list)  # [(step_name, salary), ...]

print("Data models defined")
print(" • UserProfile: User profile created from JAAT-extracted resume data")
print(" • Occupation: O*NET occupation with skills and knowledge requirements")
print(" • MarketData: Real market data (wages, demand) from DOL")
print(" • JobPosting: Job posting analysis")
print(" • CareerPathway: Step-by-step career progression path")


## 2. Data Loading & ModelsLoad real data from:- **O*NET Database**: Occupations, skills, knowledge requirements- **DOL Apprenticeship Data API**: Real market wages by occupation/state- **O*NET-SOC Code Labels**: Occupation mapping from Google Sheets

### 3A: O*NET Data Loading

In [ ]:
def load_onet_soc_labels() -> Dict[str, Dict]:"""Load O*NET-SOC code labels from Google Sheets or local file.Returns:Dict mapping SOC codes to occupation information"""# In production, this would fetch from Google Sheets# For POC, we'll use a sample structure or local fileonet_labels = {}# Try to load from local file if availabletry:# Check if there's a local file in ops/fixtures/onet/onet_dir = Path("ops/fixtures/onet")if onet_dir.exists():for json_file in onet_dir.glob("*.json"):try:with open(json_file, 'r') as f:data = json_lib.load(f)# Extract SOC code and title from O*NET JSON structureif isinstance(data, dict):soc_code = data.get('code', '')title = data.get('title', '')if soc_code and title:onet_labels[soc_code] = {'title': title,'description': data.get('description', ''),'category': data.get('category', '')}except Exception as e:print(f"Warning: Could not load {json_file}: {e}")except Exception as e:print(f"Note: Could not load local O*NET files: {e}")if not onet_labels:print("Note: No O*NET labels loaded. Using sample data structure.")# Sample structure for demonstrationonet_labels = {"15-1132.00": {"title": "Software Developers, Applications", "description": "Develop, create, and modify general computer applications software."},"15-1133.00": {"title": "Software Developers, Systems Software", "description": "Research, design, develop, and test operating systems-level software."},"15-1142.00": {"title": "Network and Computer Systems Administrators", "description": "Install, configure, and support an organization's local area network (LAN)."},}print(f"Loaded {len(onet_labels)} O*NET occupation labels")return onet_labelsdef load_sample_job_titles() -> pd.DataFrame:"""Load sample reported job titles CSV for validation.Returns:DataFrame with job titles"""# In production, this would load from a CSV file# For POC, return empty DataFrame or sample datatry:# Try to load from a local filecsv_path = Path("data/sample_job_titles.csv")if csv_path.exists():return pd.read_csv(csv_path)except Exception as e:print(f"Note: Could not load sample job titles: {e}")# Return sample structurereturn pd.DataFrame({'title': ['Software Engineer', 'Data Scientist', 'Product Manager'],'soc_code': ['15-1132.00', '15-2041.00', '11-9199.00']})def download_onet_database(format='json') -> Dict:"""Download O*NET database from onetcenter.org.Args:format: 'json' or 'xml'Returns:Dict containing occupation data"""print("Note: O*NET database download would fetch from onetcenter.org")print("For POC, using sample occupation data structure")# In production, this would:# 1. Download from https://www.onetcenter.org/database.html# 2. Parse XML/JSON files# 3. Extract occupations, skills, knowledge requirements# Sample structure for demonstrationoccupations_data = {"15-1132.00": {"title": "Software Developers, Applications","description": "Develop, create, and modify general computer applications software.","tasks": ["Modify existing software to correct errors","Design and develop software systems","Analyze user needs and software requirements"],"skills": {"programming": 5,"problem_solving": 5,"software_design": 4,"testing": 4,"documentation": 3},"knowledge": {"computers_and_electronics": 5,"mathematics": 4,"engineering_and_technology": 4},"education": "Bachelor's degree"}}return occupations_data# Load O*NET dataprint("="*80)print("LOADING O*NET DATA")print("="*80)onet_labels = load_onet_soc_labels()sample_job_titles = load_sample_job_titles()onet_database = download_onet_database()print(f"\nLoaded {len(onet_labels)} O*NET occupation labels")print(f"Loaded {len(sample_job_titles)} sample job titles")print(f"Loaded {len(onet_database)} occupations from O*NET database")print("="*80)


### 3B: DOL Market Data Loading

In [ ]:
def load_dol_apprenticeship_data(cache=True) -> pd.DataFrame:"""Fetch DOL Apprenticeship Data from API.API: apiprod.dol.gov/v4/get/ETA/apprenticeship_data/jsonArgs:cache: If True, cache data locally for offline useReturns:DataFrame with apprenticeship data (occupation, state, wages, demographics)"""cache_file = Path("data/dol_apprenticeship_cache.json")# Try to load from cache firstif cache and cache_file.exists():try:print("Loading DOL data from cache...")with open(cache_file, 'r') as f:data = json_lib.load(f)df = pd.DataFrame(data)print(f"Loaded {len(df)} records from cache")return dfexcept Exception as e:print(f"Warning: Could not load cache: {e}")# Fetch from APIprint("Fetching DOL Apprenticeship Data from API...")api_url = "https://apiprod.dol.gov/v4/get/ETA/apprenticeship_data/json"try:response = requests.get(api_url, timeout=30)response.raise_for_status()data = response.json()# Convert to DataFrameif isinstance(data, list):df = pd.DataFrame(data)elif isinstance(data, dict) and 'data' in data:df = pd.DataFrame(data['data'])else:df = pd.DataFrame([data])# Cache the dataif cache:cache_file.parent.mkdir(parents=True, exist_ok=True)df.to_json(cache_file, orient='records')print(f"Cached {len(df)} records to {cache_file}")print(f"Fetched {len(df)} records from DOL API")return dfexcept Exception as e:print(f"WARNING: Warning: Could not fetch from DOL API: {e}")print("Using sample data for demonstration")# Return sample data structuresample_data = {'occupation': ['Software Developer', 'Data Scientist', 'Network Administrator'],'state': ['CA', 'NY', 'TX'],'starting_wage': [65000, 70000, 60000],'exit_wage': [95000, 110000, 85000],'median_wage': [80000, 90000, 72500]}return pd.DataFrame(sample_data)def get_market_salary_by_occupation(title: str, state: str = None) -> Dict:"""Query cached DOL data for market salary by occupation.Args:title: Job title or occupation namestate: Optional state code (e.g., 'CA', 'NY')Returns:Dict with salary information (median, p10, p90, starting, exit)"""dol_data = load_dol_apprenticeship_data(cache=True)# Filter by occupation titleif 'occupation' in dol_data.columns:matches = dol_data[dol_data['occupation'].str.contains(title, case=False, na=False)]else:matches = dol_data# Filter by state if providedif state and 'state' in matches.columns:matches = matches[matches['state'] == state]if len(matches) == 0:return {'median_salary': None,'starting_wage': None,'exit_wage': None,'state': state}# Calculate statisticsresult = {'median_salary': matches.get('median_wage', pd.Series()).median() if 'median_wage' in matches.columns else None,'starting_wage': matches.get('starting_wage', pd.Series()).median() if 'starting_wage' in matches.columns else None,'exit_wage': matches.get('exit_wage', pd.Series()).median() if 'exit_wage' in matches.columns else None,'state': state}return result# Load DOL market dataprint("="*80)print("LOADING DOL MARKET DATA")print("="*80)dol_data = load_dol_apprenticeship_data(cache=True)print(f"\nLoaded {len(dol_data)} DOL apprenticeship records")if len(dol_data) > 0:print(f"Columns: {', '.join(dol_data.columns[:5].tolist())}")print("="*80)


## 3. Resume Upload & Processing**All data comes from your resume** - extracted using JAAT TaskMatch.This notebook demonstrates:- **JAAT TaskMatch**: Extracts skills and feature vectors from your resume text- **JAAT TitleMatch**: Matches job titles to standardized codes (if present in resume)- **Resume Parsing**: Extracts experience, education, and other metadata- **Occupation Matching**: Validates extracted job titles against O*NET-SOC codes- **Initial Match Scores**: Compares user skills with O*NET occupation requirements**No external databases or mock data** - everything is extracted from your resume using real JAAT NLP models.

In [ ]:
def parse_resume_text(resume_text: str) -> Dict:"""Parse resume text to extract key information."""text_lower = resume_text.lower()info = {'years_experience': 0,'education': "Bachelor's degree",'job_titles': []}import reexp_patterns = [r'(\d+)\+?\s*years?\s*(?:of\s*)?(?:professional\s*)?experience',r'(?:professional|work|relevant)\s*experience[:\s]+(?:of\s*)?(\d+)\+?\s*years?',r'experience[:\s]+(?:of\s*)?(\d+)\+?\s*years?',r'(\d+)\+?\s*years?\s*(?:of\s*)?(?:experience|in|working)',r'(\d+)\+?\s*years?\s*in\s*(?:the\s*)?(?:field|industry|profession)',]for pattern in exp_patterns:match = re.search(pattern, text_lower)if match:years = int(match.group(1))info['years_experience'] = yearsbreakeducation_section = ""lines = resume_text.split('\n')for i, line in enumerate(lines):if 'education' in line.lower() and len(line.strip()) < 50:education_section = '\n'.join(lines[i:min(i+20, len(lines))]).lower()breaksearch_text = education_section if education_section else text_lowerphd_patterns = [r'\bph\.?\s*d\.?\b',r'\bdoctorate\b',r'\bdoctoral\s+degree\b',r'\bdoctor\s+of\s+philosophy\b',]if any(re.search(pattern, search_text) for pattern in phd_patterns):info['education'] = "Doctorate"elif re.search(r'\bmaster\s+(?:of|in|degree)', search_text) or \re.search(r'\bmaster\'?s?\s+degree\b', search_text) or \re.search(r'\b(?:m\.?\s*s\.?\s*c?\.?|msc|mba)\s+(?:in|of|degree)\b', search_text) or \re.search(r'\b(?:mba|msc|m\.?\s*b\.?\s*a\.?)\b', search_text) or \re.search(r'\bm\.?\s*s\.?\s*c?\.?\s+(?:in|of|degree)\b', search_text):info['education'] = "Master's degree"elif re.search(r'\bbachelor\s+(?:of|in|degree)', search_text) or \re.search(r'\b(?:b\.?\s*s\.?\s*c?\.?|bsc|b\.?\.a\.?|ba)\b', search_text) or \re.search(r'\bbachelor\'?s?\b', search_text):info['education'] = "Bachelor's degree"elif re.search(r'\bassociate\s+(?:of|in|degree)', search_text) or \re.search(r'\b(?:a\.?\s*a\.?|a\.?\s*s\.?)\b', search_text):info['education'] = "Associate's degree"elif re.search(r'\b(?:certificate|diploma)\s+(?:in|of)', search_text):info['education'] = "Certificate"title_keywords = ['developer', 'engineer', 'programmer', 'analyst', 'manager', 'scientist','designer', 'architect', 'consultant', 'specialist', 'coordinator']for keyword in title_keywords:if keyword in text_lower:pattern = rf'(\w+\s+)?{keyword}(\s+\w+)?'matches = re.findall(pattern, text_lower)if matches:for m in matches:full_title = (m[0] or '') + keyword + (m[1] or '')if full_title not in info['job_titles']:info['job_titles'].append(full_title)return infodef create_user_profile_from_resume(resume_text: str, jaat_features) -> UserProfile:"""Create a UserProfile from resume text and JAAT-extracted features."""resume_info = parse_resume_text(resume_text)user_skills = {}if isinstance(jaat_features, dict):for jaat_feature, weight in jaat_features.items():skill_name = jaat_feature.replace('textual_skill_', '').replace('_', ' ').title()if isinstance(weight, (int, float)):if weight <= 1.0:proficiency = max(1, min(5, int(weight * 5)))else:proficiency = max(1, min(5, int(weight)))else:proficiency = 3user_skills[skill_name] = proficiency# Try to match job titles to O*NET occupationscurrent_occupation = Noneif resume_info['job_titles'] and TiM:try:title_matches = TiM.get_title(resume_info['job_titles'])if title_matches and len(title_matches) > 0:# Get the first matched SOC codeif isinstance(title_matches[0], (list, tuple)) and len(title_matches[0]) > 0:current_occupation = str(title_matches[0][0]) if isinstance(title_matches[0][0], (str, int)) else Noneelif isinstance(title_matches[0], dict) and 'soc_code' in title_matches[0]:current_occupation = title_matches[0]['soc_code']except Exception as e:print(f"Note: Could not match job titles: {e}")profile = UserProfile(user_id='resume_user',skills=user_skills,experience_years=resume_info['years_experience'] or 2,education_level=resume_info['education'],jaat_feature_weights=jaat_features,current_occupation=current_occupation)return profiledef extract_text_from_pdf(pdf_bytes):"""Extract text from PDF file bytes."""if PDF_LIBRARY == 'PyPDF2':try:import iopdf_file = io.BytesIO(pdf_bytes)pdf_reader = PyPDF2.PdfReader(pdf_file)text = ""for page in pdf_reader.pages:text += page.extract_text() + "\n"return textexcept Exception as e:print(f"WARNING: Error extracting text with PyPDF2: {e}")return Noneelif PDF_LIBRARY == 'pdfplumber':try:import iopdf_file = io.BytesIO(pdf_bytes)text = ""with pdfplumber.open(pdf_file) as pdf:for page in pdf.pages:text += page.extract_text() + "\n"return textexcept Exception as e:print(f"WARNING: Error extracting text with pdfplumber: {e}")return Noneelse:return Nonedef ensure_jaat_initialized():"""Ensure JAAT is initialized."""global SM, TM, TiMtry:_ = SMsm_initialized = SM is not Noneexcept NameError:sm_initialized = Falsetry:_ = TMtm_initialized = TM is not Noneexcept NameError:tm_initialized = Falsetry:_ = TiMtim_initialized = TiM is not Noneexcept NameError:tim_initialized = Falseif not (sm_initialized or tm_initialized):print("\nWARNING: JAAT not initialized. Please run Section 1 first.")raise RuntimeError("JAAT not initialized")if not tim_initialized:raise RuntimeError("TitleMatch is not initialized")return SM, TM, TiMprint("Resume processing functions defined")


In [ ]:
def process_resume_with_jaat(resume_text):"""Process resume text with JAAT and create user profile with occupation matching."""global resume_user, jaat_successresume_to_process = resume_text.strip()if len(resume_to_process) < 50:print("WARNING: Resume text is too short. Please provide a complete resume.")returnprint("="*80)print("RESUME UPLOAD & JAAT EXTRACTION")print("="*80)print(f"\nProcessing your resume ({len(resume_to_process)} characters)")try:SM, TM, TiM = ensure_jaat_initialized()except RuntimeError as e:print(f"\nERROR: ERROR: {e}")returnprint("\nStep 1: Using REAL JAAT to extract skills from your resume...")import osos.environ['OMP_NUM_THREADS'] = '1'os.environ['TOKENIZERS_PARALLELISM'] = 'false'jaat_extracted = Nonejaat_success = Falseif SM:try:print(" Trying SkillMatch (preferred for resume skill extraction)...")if hasattr(SM, 'get_skills_batch'):try:skills_list = SM.get_skills_batch([resume_to_process])print(f"Used SM.get_skills_batch([text]) - SUCCESS")if isinstance(skills_list, list) and len(skills_list) > 0:if isinstance(skills_list[0], list):skills_list = skills_list[0]jaat_extracted = {}for i, skill_item in enumerate(skills_list):if isinstance(skill_item, tuple) and len(skill_item) >= 2:skill_label, europa_code = skill_item[0], skill_item[1]feature_name = f"textual_skill_{skill_label.lower().replace(' ', '_')}"jaat_extracted[feature_name] = 1.0 - (i * 0.05) if i < 20 else 0.05jaat_success = Trueprint(f"Converted {len(skills_list)} skills to feature vectors")except Exception as e:print(f"Error with get_skills_batch: {type(e).__name__}: {str(e)[:150]}")if jaat_extracted is None and hasattr(SM, 'get_skills'):try:skills_list = SM.get_skills(resume_to_process)print(f"Used SM.get_skills(text) - SUCCESS")if isinstance(skills_list, list) and len(skills_list) > 0:jaat_extracted = {}for i, skill_item in enumerate(skills_list):if isinstance(skill_item, tuple) and len(skill_item) >= 2:skill_label, europa_code = skill_item[0], skill_item[1]feature_name = f"textual_skill_{skill_label.lower().replace(' ', '_')}"jaat_extracted[feature_name] = 1.0 - (i * 0.05) if i < 20 else 0.05jaat_success = Trueprint(f"Converted {len(skills_list)} skills to feature vectors")except Exception as e:print(f"Error with get_skills: {type(e).__name__}: {str(e)[:150]}")except Exception as e:print(f"WARNING: WARNING: Unexpected error with SkillMatch: {type(e).__name__}: {e}")if jaat_extracted is None and TM:try:print(" Trying TaskMatch as fallback (extracts O*NET tasks, not skills)...")if hasattr(TM, 'get_tasks_batch'):try:tasks_list = TM.get_tasks_batch([resume_to_process])print(f"Used TM.get_tasks_batch([text]) - SUCCESS")if isinstance(tasks_list, list) and len(tasks_list) > 0:if isinstance(tasks_list[0], list):tasks_list = tasks_list[0]jaat_extracted = {}for i, task_item in enumerate(tasks_list):if isinstance(task_item, tuple) and len(task_item) >= 2:task_id, task_desc = task_item[0], task_item[1]feature_name = f"task_{task_id}" if task_id else f"task_{i}"jaat_extracted[feature_name] = 1.0 - (i * 0.05) if i < 20 else 0.05print(f"Converted {len(tasks_list)} tasks to feature vectors")except Exception as e:print(f"Error with get_tasks_batch: {type(e).__name__}: {str(e)[:150]}")if jaat_extracted is None and hasattr(TM, 'get_tasks'):try:tasks_list = TM.get_tasks(resume_to_process)print(f"Used TM.get_tasks(text) - SUCCESS")if isinstance(tasks_list, list) and len(tasks_list) > 0:jaat_extracted = {}for i, task_item in enumerate(tasks_list):if isinstance(task_item, tuple) and len(task_item) >= 2:task_id, task_desc = task_item[0], task_item[1]feature_name = f"task_{task_id}" if task_id else f"task_{i}"jaat_extracted[feature_name] = 1.0 - (i * 0.05) if i < 20 else 0.05jaat_success = Trueprint(f"Converted {len(tasks_list)} tasks to feature vectors")except Exception as e:print(f"Error with get_tasks: {type(e).__name__}: {str(e)[:150]}")except Exception as e:print(f"WARNING: WARNING: Unexpected error with TaskMatch: {type(e).__name__}: {e}")if not jaat_extracted or (isinstance(jaat_extracted, dict) and len(jaat_extracted) == 0):raise RuntimeError("JAAT failed to extract any skills from the resume.")print("\nStep 2: Creating user profile from JAAT-extracted data...")if isinstance(jaat_extracted, dict):jaat_features = jaat_extractedelif isinstance(jaat_extracted, list):jaat_features = {f"jaat_feature_{i}": val for i, val in enumerate(jaat_extracted)}else:jaat_features = {}resume_info = parse_resume_text(resume_to_process)# Step 3: Validate job titles against O*NET-SOC codesprint("\nStep 3: Validating job titles against O*NET-SOC codes...")matched_occupations = []if resume_info['job_titles'] and TiM:try:title_matches = TiM.get_title(resume_info['job_titles'])print(f"Matched {len(title_matches) if title_matches else 0} job titles to standardized codes")if title_matches:for match in title_matches[:3]: # Top 3 matchesif isinstance(match, (list, tuple)) and len(match) > 0:soc_code = str(match[0]) if match[0] else Noneif soc_code and soc_code in onet_labels:matched_occupations.append({'soc_code': soc_code,'title': onet_labels[soc_code]['title']})except Exception as e:print(f"WARNING: TitleMatch error: {e}")resume_user = create_user_profile_from_resume(resume_to_process, jaat_features)print(f"\nProfile created:")print(f"• Experience: {resume_user.experience_years} years")print(f"• Education: {resume_user.education_level}")print(f"• Skills extracted: {len(resume_user.skills)}")if matched_occupations:print(f"• Matched occupations: {len(matched_occupations)}")for occ in matched_occupations[:3]:print(f"- {occ['title']} ({occ['soc_code']})")print("\n" + "="*80)print("RESUME PROCESSING COMPLETE!")print("="*80)# File upload handler (simplified version - full version would be in separate cell)def handle_upload(change):"""Handle file upload and process resume."""try:uploaded_file = change['owner']file_data_input = change.get('new') or uploaded_file.valueif not file_data_input:returnoutput.clear_output()file_name = Nonefile_data = Noneif isinstance(file_data_input, dict):file_name = list(file_data_input.keys())[0]file_data = file_data_input[file_name]elif isinstance(file_data_input, tuple) and len(file_data_input) >= 2:file_name, file_data = file_data_input[0], file_data_input[1]if not file_data or 'content' not in file_data:returnfile_content = file_data['content']if isinstance(file_content, memoryview):file_content = file_content.tobytes()file_type = file_name.split('.')[-1].lower() if file_name else 'unknown'with output:resume_text = Noneif file_type == 'pdf':resume_text = extract_text_from_pdf(file_content)elif file_type == 'txt':resume_text = file_content.decode('utf-8')if resume_text:process_resume_with_jaat(resume_text)except Exception as e:output.clear_output()with output:print(f"WARNING: ERROR: {e}")if IPYWIDGETS_AVAILABLE:print("="*80)print("RESUME UPLOAD - PDF FILE UPLOAD")print("="*80)print("\nUpload your resume PDF file:")print(" 1. Click the 'Upload' button below")print(" 2. Select your resume PDF file")print(" 3. The file will be automatically processed")print("\n" + "-"*80)upload = FileUpload(accept='.pdf,.txt',multiple=False,description='Upload Resume PDF')output = Output()upload.observe(handle_upload, names='value')print("\n" + "="*80)print("FILE UPLOAD WIDGET")print("="*80)print("\nClick the button below to upload your resume PDF:")display(upload)display(output)print("\n" + "-"*80)print("OR: Paste your resume text manually below (Option 2)")print("-"*80)else:print("="*80)print("RESUME UPLOAD")print("="*80)print("\nFile upload not available. Please paste your resume text below.")print("To enable file upload, install: pip install ipywidgets")print("-"*80)


## 4. Skill Analysis from ResumeAnalyze the skills extracted from your resume by JAAT TaskMatch.

In [ ]:
def analyze_extracted_skills(user: UserProfile) -> Dict:"""Analyze skills extracted from resume by JAAT TaskMatch."""top_skills = sorted(user.skills.items(),key=lambda x: x[1],reverse=True) if user.skills else []return {'total_skills': len(user.skills),'experience_years': user.experience_years,'education_level': user.education_level,'top_skills': top_skills,'feature_vectors': user.jaat_feature_weights or {}}def verify_resume_uploaded():"""Verify if a resume has been successfully uploaded and processed."""if 'resume_user' not in globals():return Falseif not resume_user or not isinstance(resume_user, UserProfile):return Falsereturn Trueprint("="*80)print("SKILL ANALYSIS: Resume-Extracted Profile")print("="*80)if not verify_resume_uploaded():print("\nWARNING: RESUME NOT FOUND!")print(" Your resume has not been uploaded yet.")print(" Please go back to Section 3 and upload your resume first.")print("="*80)else:skill_analysis = analyze_extracted_skills(resume_user)print(f"\nSKILL SUMMARY (Extracted by JAAT)")print("="*80)print(f"Total Skills Extracted: {skill_analysis['total_skills']}")print(f"Experience: {skill_analysis['experience_years']} years")print(f"Education: {skill_analysis['education_level']}")if skill_analysis['top_skills']:print(f"\nTop Skills (by proficiency):")for i, (skill_name, proficiency) in enumerate(skill_analysis['top_skills'], 1):print(f"{i}. {skill_name}: {proficiency}/5")if skill_analysis['feature_vectors']:print(f"\nJAAT Feature Vectors (from NLx corpus):")sorted_features = sorted(skill_analysis['feature_vectors'].items(),key=lambda x: x[1] if isinstance(x[1], (int, float)) else 0,reverse=True)for feature, weight in sorted_features[:10]:print(f"• {feature}: {weight}")print("\n" + "="*80)


## 5. Job Posting AnalysisUpload or paste a job posting to analyze required skills and match against your resume.

In [ ]:
def analyze_job_posting_with_jaat(job_text: str) -> JobPosting:"""Analyze job posting using JAAT TaskMatch/SkillMatch and TitleMatch."""print("Analyzing job posting with JAAT...")# Extract skills using JAATrequired_skills = {}if SM:try:if hasattr(SM, 'get_skills'):skills_list = SM.get_skills(job_text)if isinstance(skills_list, list):for i, skill_item in enumerate(skills_list):if isinstance(skill_item, tuple) and len(skill_item) >= 2:skill_label = skill_item[0]importance = 5 - min(i, 4) # First skills are more importantrequired_skills[skill_label.lower().replace(' ', '_')] = importanceexcept Exception as e:print(f"Warning: Could not extract skills: {e}")# Match job title to occupationmatched_occupation_code = Noneif TiM:try:# Extract job title from posting (first line or title section)lines = job_text.split('\n')potential_title = lines[0] if lines else job_text[:100]title_matches = TiM.get_title([potential_title])if title_matches and len(title_matches) > 0:if isinstance(title_matches[0], (list, tuple)) and len(title_matches[0]) > 0:matched_occupation_code = str(title_matches[0][0])except Exception as e:print(f"Warning: Could not match title: {e}")# Extract salary range (simple regex)import resalary_range = Nonesalary_patterns = [r'\$(\d{1,3}(?:,\d{3})*(?:k|K)?)\s*-\s*\$(\d{1,3}(?:,\d{3})*(?:k|K)?)',r'(\d{1,3}(?:,\d{3})*(?:k|K)?)\s*-\s*(\d{1,3}(?:,\d{3})*(?:k|K)?)\s*(?:salary|compensation)',]for pattern in salary_patterns:match = re.search(pattern, job_text, re.IGNORECASE)if match:try:low = int(match.group(1).replace(',', '').replace('k', '000').replace('K', '000'))high = int(match.group(2).replace(',', '').replace('k', '000').replace('K', '000'))salary_range = (low, high)breakexcept:passreturn JobPosting(title=potential_title if 'potential_title' in locals() else "Unknown",description=job_text,required_skills=required_skills,salary_range=salary_range,matched_occupation_code=matched_occupation_code)def match_resume_to_job_posting(user: UserProfile, job: JobPosting) -> Dict:"""Match user resume to job posting and calculate match score."""user_skill_names = {k.lower().replace(' ', '_'): v for k, v in user.skills.items()}job_skill_names = {k.lower().replace(' ', '_'): v for k, v in job.required_skills.items()}matched_skills = []missing_skills = []for job_skill, importance in job_skill_names.items():user_proficiency = user_skill_names.get(job_skill, 0)if user_proficiency > 0:matched_skills.append({'skill': job_skill,'user_proficiency': user_proficiency,'required_importance': importance})else:missing_skills.append({'skill': job_skill,'required_importance': importance})total_required = len(job_skill_names)matched_count = len(matched_skills)match_score = (matched_count / total_required * 100) if total_required > 0 else 0return {'match_score': match_score,'matched_skills': matched_skills,'missing_skills': missing_skills,'total_required': total_required,'matched_count': matched_count}# Job posting inputprint("="*80)print("JOB POSTING ANALYSIS")print("="*80)print("\nPaste a job posting below to analyze required skills:")print("(In production, this would have a file upload widget)")# Example job posting for demonstrationEXAMPLE_JOB_POSTING = """Software Engineer - Full Stack DeveloperWe are looking for a skilled Software Engineer with experience in:- Python programming (3+ years)- React and JavaScript- Database design (PostgreSQL)- RESTful API development- Cloud platforms (AWS)Salary: $90,000 - $120,000Location: San Francisco, CA"""print("\nExample job posting loaded. Set YOUR_JOB_POSTING variable to analyze your own posting.")print("="*80)


## 6. Skill-to-Occupation MatchingMatch your user profile to O*NET occupations based on skills, experience, and education.

In [ ]:
def create_occupation_from_onet(soc_code: str, onet_data: Dict) -> Occupation:"""Create Occupation object from O*NET data."""occ_data = onet_data.get(soc_code, {})return Occupation(soc_code=soc_code,title=occ_data.get('title', 'Unknown'),description=occ_data.get('description', ''),required_skills=occ_data.get('skills', {}),required_knowledge=occ_data.get('knowledge', {}),education_level=occ_data.get('education', "Bachelor's degree"),tasks=occ_data.get('tasks', []))def match_user_to_occupations(user: UserProfile, occupations: List[Occupation]) -> List[Tuple[Occupation, float]]:"""Match user profile to occupations and return sorted list of (Occupation, match_score).Args:user: UserProfile from resumeoccupations: List of Occupation objectsReturns:List of (Occupation, match_score) tuples, sorted by score descending"""matches = []for occupation in occupations:matched_skills = []missing_skills = []proficiency_gaps = []# Normalize skill names for comparisonuser_skill_names = {k.lower().replace(' ', '_'): v for k, v in user.skills.items()}occ_skill_names = {k.lower().replace(' ', '_'): v for k, v in occupation.required_skills.items()}for skill_id, required_importance in occ_skill_names.items():user_proficiency = user_skill_names.get(skill_id, 0)if user_proficiency > 0:matched_skills.append({'skill_id': skill_id,'user_proficiency': user_proficiency,'required_importance': required_importance,'gap': max(0, required_importance - user_proficiency)})if user_proficiency < required_importance:proficiency_gaps.append({'skill_id': skill_id,'gap': required_importance - user_proficiency})else:missing_skills.append({'skill_id': skill_id,'required_importance': required_importance})# Calculate base match scoretotal_required = len(occ_skill_names)matched_count = len(matched_skills)base_score = (matched_count / total_required * 100) if total_required > 0 else 0# Calculate proficiency alignmentif matched_skills:avg_proficiency_alignment = sum(1 - (gap['gap'] / 5) for gap in proficiency_gaps) / len(matched_skills) if matched_skills else 0avg_proficiency_alignment = max(0, avg_proficiency_alignment)else:avg_proficiency_alignment = 0# Experience factorexperience_factor = min(1.1, 1.0 + (user.experience_years * 0.01))# Education compatibilityeducation_levels = ["Certificate", "Associate's degree", "Bachelor's degree", "Master's degree", "Doctorate"]user_edu_idx = education_levels.index(user.education_level) if user.education_level in education_levels else 2occ_edu_idx = education_levels.index(occupation.education_level) if occupation.education_level in education_levels else 2education_factor = 1.0 if user_edu_idx >= occ_edu_idx else 0.8# Final scorefinal_score = base_score * avg_proficiency_alignment * experience_factor * education_factormatches.append((occupation, final_score))# Sort by score descendingmatches.sort(key=lambda x: x[1], reverse=True)return matches# Load occupations from O*NET dataprint("="*80)print("OCCUPATION MATCHING")print("="*80)if 'resume_user' not in globals():print("\nWARNING: Please upload your resume in Section 3 first.")print("="*80)else:# Create Occupation objects from O*NET dataoccupations_list = []for soc_code, occ_data in onet_database.items():occ = create_occupation_from_onet(soc_code, onet_database)occupations_list.append(occ)# Match user to occupationsoccupation_matches = match_user_to_occupations(resume_user, occupations_list)print(f"\nTop 10 Occupation Matches:")print("="*80)for i, (occupation, score) in enumerate(occupation_matches[:10], 1):print(f"{i}. {occupation.title} ({occupation.soc_code})")print(f"Match Score: {score:.1f}%")print(f"Required Education: {occupation.education_level}")print()print("="*80)


### 6B: Occupation Matching Visualization

In [ ]:
if 'resume_user' in globals() and 'occupation_matches' in globals():# Visualize top occupation matchestop_matches = occupation_matches[:10]occ_titles = [occ.title for occ, _ in top_matches]match_scores = [score for _, score in top_matches]fig, ax = plt.subplots(figsize=(14, 8))bars = ax.barh(occ_titles, match_scores, color=sns.color_palette("viridis", len(occ_titles)))ax.set_xlabel('Match Score (%)', fontsize=12)ax.set_title('Top 10 Occupation Matches', fontsize=14, fontweight='bold')ax.set_xlim(0, 100)ax.grid(axis='x', alpha=0.3)for i, (title, score) in enumerate(zip(occ_titles, match_scores)):ax.text(score + 1, i, f"{score:.1f}%", va='center', fontsize=10)plt.tight_layout()plt.show()print(f"\nTop Match: {occ_titles[0]} ({match_scores[0]:.1f}%)")else:print("Please run Section 6 first to generate occupation matches.")


## 7. Skill Gap AnalysisIdentify missing and weak skills for target occupations.

In [ ]:
def calculate_skill_gap(user: UserProfile, occupation: Occupation) -> Dict:"""Calculate skill gap between user and target occupation.Returns:Dict with missing_skills, weak_skills, gap_severity, etc."""user_skill_names = {k.lower().replace(' ', '_'): v for k, v in user.skills.items()}occ_skill_names = {k.lower().replace(' ', '_'): v for k, v in occupation.required_skills.items()}missing_skills = []weak_skills = []strong_skills = []for skill_id, required_importance in occ_skill_names.items():user_proficiency = user_skill_names.get(skill_id, 0)if user_proficiency == 0:missing_skills.append({'skill_id': skill_id,'skill_name': skill_id.replace('_', ' ').title(),'required_importance': required_importance,'priority': 'critical' if required_importance >= 4 else 'important' if required_importance >= 3 else 'nice-to-have'})elif user_proficiency < required_importance:weak_skills.append({'skill_id': skill_id,'skill_name': skill_id.replace('_', ' ').title(),'user_proficiency': user_proficiency,'required_importance': required_importance,'gap': required_importance - user_proficiency})else:strong_skills.append({'skill_id': skill_id,'skill_name': skill_id.replace('_', ' ').title(),'user_proficiency': user_proficiency,'required_importance': required_importance})# Calculate gap severity scorecritical_missing = len([s for s in missing_skills if s['priority'] == 'critical'])important_missing = len([s for s in missing_skills if s['priority'] == 'important'])total_gap_severity = critical_missing * 3 + important_missing * 2 + len(weak_skills) * 1return {'missing_skills': sorted(missing_skills, key=lambda x: x['required_importance'], reverse=True),'weak_skills': sorted(weak_skills, key=lambda x: x['gap'], reverse=True),'strong_skills': strong_skills,'gap_severity': total_gap_severity,'critical_missing_count': critical_missing,'important_missing_count': important_missing,'weak_skills_count': len(weak_skills)}# Analyze gap for top occupation matchprint("="*80)print("SKILL GAP ANALYSIS")print("="*80)if 'resume_user' in globals() and 'occupation_matches' in globals() and len(occupation_matches) > 0:target_occupation = occupation_matches[0][0] # Top matchgap_analysis = calculate_skill_gap(resume_user, target_occupation)print(f"\nTarget Occupation: {target_occupation.title}")print(f"Gap Severity Score: {gap_analysis['gap_severity']}")print(f"\nMissing Skills: {len(gap_analysis['missing_skills'])}")print(f"• Critical: {gap_analysis['critical_missing_count']}")print(f"• Important: {gap_analysis['important_missing_count']}")print(f"• Nice-to-have: {len(gap_analysis['missing_skills']) - gap_analysis['critical_missing_count'] - gap_analysis['important_missing_count']}")print(f"\nWeak Skills (need improvement): {gap_analysis['weak_skills_count']}")print(f"Strong Skills: {len(gap_analysis['strong_skills'])}")if gap_analysis['missing_skills']:print("\nTop Missing Skills (by importance):")for skill in gap_analysis['missing_skills'][:10]:print(f"• {skill['skill_name']} (Required: {skill['required_importance']}/5, Priority: {skill['priority']})")if gap_analysis['weak_skills']:print("\nSkills Needing Improvement:")for skill in gap_analysis['weak_skills'][:10]:print(f"• {skill['skill_name']} (You: {skill['user_proficiency']}/5, Required: {skill['required_importance']}/5, Gap: {skill['gap']})")print("="*80)else:print("\nWARNING: Please run Section 6 first to generate occupation matches.")print("="*80)


### 7B: Gap Analysis Visualization

In [ ]:
if 'gap_analysis' in globals():# Create gap analysis visualizationsfig, axes = plt.subplots(1, 2, figsize=(16, 6))# Missing skills by prioritymissing_by_priority = {'Critical': len([s for s in gap_analysis['missing_skills'] if s['priority'] == 'critical']),'Important': len([s for s in gap_analysis['missing_skills'] if s['priority'] == 'important']),'Nice-to-have': len([s for s in gap_analysis['missing_skills'] if s['priority'] == 'nice-to-have'])}axes[0].bar(missing_by_priority.keys(), missing_by_priority.values(),color=['#d62728', '#ff7f0e', '#2ca02c'])axes[0].set_ylabel('Number of Skills', fontsize=12)axes[0].set_title('Missing Skills by Priority', fontsize=14, fontweight='bold')axes[0].grid(axis='y', alpha=0.3)# Skills needing improvementif gap_analysis['weak_skills']:weak_skill_names = [s['skill_name'] for s in gap_analysis['weak_skills'][:10]]weak_skill_gaps = [s['gap'] for s in gap_analysis['weak_skills'][:10]]axes[1].barh(weak_skill_names, weak_skill_gaps, color=sns.color_palette("YlOrRd", len(weak_skill_names)))axes[1].set_xlabel('Proficiency Gap', fontsize=12)axes[1].set_title('Top Skills Needing Improvement', fontsize=14, fontweight='bold')axes[1].grid(axis='x', alpha=0.3)else:axes[1].text(0.5, 0.5, 'No weak skills found',ha='center', va='center', fontsize=12)axes[1].set_title('Skills Needing Improvement', fontsize=14, fontweight='bold')plt.tight_layout()plt.show()else:print("Please run Section 7 first to generate gap analysis.")


## 8. Market Data IntegrationView real market salaries, demand trends, and skills demand analysis.

In [ ]:
def get_market_salary(occupation_code: str, state: str = None) -> MarketData:"""Get market salary data for an occupation from DOL data."""# Try to find occupation title from O*NEToccupation_title = Noneif occupation_code in onet_labels:occupation_title = onet_labels[occupation_code]['title']else:occupation_title = "Unknown Occupation"# Query DOL datasalary_info = get_market_salary_by_occupation(occupation_title, state)return MarketData(occupation_code=occupation_code,median_salary=salary_info.get('median_salary'),starting_wage=salary_info.get('starting_wage'),exit_wage=salary_info.get('exit_wage'),state=state)def analyze_demand_trends(occupation_code: str, date_range: Tuple[str, str] = None) -> Dict:"""Analyze demand trends for an occupation.For POC, uses DOL Apprenticeship Data as proxy.In production, would use NLx aggregated datasets if available."""# Query DOL data for this occupationoccupation_title = Noneif occupation_code in onet_labels:occupation_title = onet_labels[occupation_code]['title']if occupation_title:# Count apprenticeships as proxy for demandmatches = dol_data[dol_data['occupation'].str.contains(occupation_title, case=False, na=False)] if 'occupation' in dol_data.columns else dol_data.iloc[:0]demand_count = len(matches)# Simple trend calculation (in production, would use time series)if demand_count > 10:trend = "growing"elif demand_count > 5:trend = "stable"else:trend = "declining"return {'occupation_code': occupation_code,'demand_trend': trend,'demand_count': demand_count,'growth_rate': demand_count * 0.1 # Placeholder}return {'occupation_code': occupation_code,'demand_trend': 'unknown','demand_count': 0,'growth_rate': 0}# Market data analysisprint("="*80)print("MARKET DATA INTEGRATION")print("="*80)if 'occupation_matches' in globals() and len(occupation_matches) > 0:target_occupation = occupation_matches[0][0]market_data = get_market_salary(target_occupation.soc_code)demand_trends = analyze_demand_trends(target_occupation.soc_code)print(f"\nOccupation: {target_occupation.title} ({target_occupation.soc_code})")print("\nSalary Data (from DOL Apprenticeship Data):")if market_data.median_salary:print(f"• Median Salary: ${market_data.median_salary:,.0f}")if market_data.starting_wage:print(f"• Starting Wage: ${market_data.starting_wage:,.0f}")if market_data.exit_wage:print(f"• Exit Wage: ${market_data.exit_wage:,.0f}")if market_data.starting_wage:growth = ((market_data.exit_wage - market_data.starting_wage) / market_data.starting_wage) * 100print(f"• Salary Growth: {growth:.1f}%")print("\nDemand Trends:")print(f"• Trend: {demand_trends['demand_trend'].upper()}")print(f"• Demand Count: {demand_trends['demand_count']}")if demand_trends['growth_rate'] > 0:print(f"• Growth Rate: {demand_trends['growth_rate']:.1f}%")# Visualize salary trajectoryif market_data.starting_wage and market_data.exit_wage:fig, ax = plt.subplots(figsize=(10, 6))stages = ['Entry', 'Mid', 'Senior']salaries = [market_data.starting_wage,(market_data.starting_wage + market_data.exit_wage) / 2,market_data.exit_wage]ax.plot(stages, salaries, marker='o', linewidth=2, markersize=10, color='#2ca02c')ax.fill_between(stages, salaries, alpha=0.3, color='#2ca02c')ax.set_ylabel('Salary ($)', fontsize=12)ax.set_title(f'Salary Trajectory: {target_occupation.title}', fontsize=14, fontweight='bold')ax.grid(alpha=0.3)for stage, salary in zip(stages, salaries):ax.text(stage, salary, f'${salary:,.0f}', ha='center', va='bottom', fontsize=11)plt.tight_layout()plt.show()print("="*80)else:print("\nWARNING: Please run Section 6 first to generate occupation matches.")print("="*80)


## 9. Career Pathway PlanningGenerate step-by-step career progression paths from your current role to target occupations.

In [ ]:
def find_transitional_occupations(source: Occupation, target: Occupation, all_occupations: List[Occupation]) -> List[Occupation]:"""Find transitional occupations between source and target.Looks for occupations that share skills with both source and target."""source_skills = set(source.required_skills.keys())target_skills = set(target.required_skills.keys())transitional = []for occ in all_occupations:if occ.soc_code == source.soc_code or occ.soc_code == target.soc_code:continueocc_skills = set(occ.required_skills.keys())# Calculate overlap with source and targetsource_overlap = len(source_skills & occ_skills) / len(source_skills) if source_skills else 0target_overlap = len(target_skills & occ_skills) / len(target_skills) if target_skills else 0# Good transitional if it has overlap with bothif source_overlap > 0.3 and target_overlap > 0.3:transitional.append((occ, source_overlap + target_overlap))# Sort by total overlaptransitional.sort(key=lambda x: x[1], reverse=True)return [occ for occ, _ in transitional[:2]] # Top 2 transitionaldef generate_career_pathway(user: UserProfile, target_occupation: Occupation, all_occupations: List[Occupation]) -> CareerPathway:"""Generate career pathway from user's current occupation to target."""# Find current occupationcurrent_occupation = Noneif user.current_occupation:for occ in all_occupations:if occ.soc_code == user.current_occupation:current_occupation = occbreak# If no current occupation found, use first match from occupation matchingif not current_occupation and 'occupation_matches' in globals() and len(occupation_matches) > 0:current_occupation = occupation_matches[0][0]if not current_occupation:# Create a generic current occupation from user skillscurrent_occupation = Occupation(soc_code="CURRENT",title="Current Role",description="Your current role based on resume",required_skills=user.skills,education_level=user.education_level)# Find transitional occupationstransitional = find_transitional_occupations(current_occupation, target_occupation, all_occupations)# Build pathway stepssteps = []# Step 1: Currentsteps.append({'occupation': current_occupation,'required_skills': {},'estimated_time': 0,'step_name': 'Current Role'})# Step 2-N: Transitional occupationsfor i, trans_occ in enumerate(transitional, 1):# Skills needed to transitionneeded_skills = {}for skill_id, importance in trans_occ.required_skills.items():if skill_id not in current_occupation.required_skills:needed_skills[skill_id] = importance# Estimate time based on number of new skillsestimated_time = len(needed_skills) * 2 # 2 months per skillsteps.append({'occupation': trans_occ,'required_skills': needed_skills,'estimated_time': estimated_time,'step_name': f'Transition {i}: {trans_occ.title}'})# Final step: Targetprev_occ = transitional[-1] if transitional else current_occupationneeded_skills = {}for skill_id, importance in target_occupation.required_skills.items():if skill_id not in prev_occ.required_skills:needed_skills[skill_id] = importanceestimated_time = len(needed_skills) * 2steps.append({'occupation': target_occupation,'required_skills': needed_skills,'estimated_time': estimated_time,'step_name': f'Target: {target_occupation.title}'})# Calculate salary trajectorysalary_trajectory = []for step in steps:if step['occupation'].soc_code != "CURRENT":market_data = get_market_salary(step['occupation'].soc_code)salary = market_data.median_salary or market_data.starting_wage or 70000salary_trajectory.append((step['step_name'], salary))total_time = sum(step['estimated_time'] for step in steps)return CareerPathway(current_occupation=current_occupation,target_occupation=target_occupation,steps=steps,total_estimated_time=total_time,salary_trajectory=salary_trajectory)# Generate career pathwayprint("="*80)print("CAREER PATHWAY PLANNING")print("="*80)if 'resume_user' in globals() and 'occupation_matches' in globals() and len(occupation_matches) > 0:target_occ = occupation_matches[0][0] # Top match as targetall_occupations = [occ for occ, _ in occupation_matches[:20]] # Use top 20 for pathway generationpathway = generate_career_pathway(resume_user, target_occ, all_occupations)print(f"\nCareer Pathway: {pathway.current_occupation.title} → {pathway.target_occupation.title}")print(f"Total Estimated Time: {pathway.total_estimated_time} months")print("\nPathway Steps:")print("="*80)cumulative_time = 0for i, step in enumerate(pathway.steps, 1):cumulative_time += step['estimated_time']print(f"\nStep {i}: {step['step_name']}")print(f"Occupation: {step['occupation'].title}")if step['required_skills']:print(f"Skills to Acquire: {len(step['required_skills'])}")for skill_id, importance in list(step['required_skills'].items())[:5]:print(f"• {skill_id.replace('_', ' ').title()} (importance: {importance}/5)")print(f"Estimated Time: {step['estimated_time']} months")print(f"Cumulative Time: {cumulative_time} months")if pathway.salary_trajectory:print("\nSalary Trajectory:")for step_name, salary in pathway.salary_trajectory:print(f"• {step_name}: ${salary:,.0f}")print("="*80)else:print("\nWARNING: Please run Section 6 first to generate occupation matches.")print("="*80)


### 9B: Pathway Visualization

In [ ]:
if 'pathway' in globals():# Visualize career pathwayfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))# Pathway timelinestep_names = [step['step_name'] for step in pathway.steps]cumulative_times = []time_so_far = 0for step in pathway.steps:time_so_far += step['estimated_time']cumulative_times.append(time_so_far)ax1.plot(cumulative_times, range(len(step_names)), marker='o', linewidth=2, markersize=10, color='#1f77b4')ax1.set_yticks(range(len(step_names)))ax1.set_yticklabels(step_names)ax1.set_xlabel('Time (months)', fontsize=12)ax1.set_title('Career Pathway Timeline', fontsize=14, fontweight='bold')ax1.grid(alpha=0.3)ax1.invert_yaxis()# Salary trajectoryif pathway.salary_trajectory:step_names_salary = [name for name, _ in pathway.salary_trajectory]salaries = [sal for _, sal in pathway.salary_trajectory]ax2.plot(range(len(step_names_salary)), salaries, marker='o', linewidth=2, markersize=10, color='#2ca02c')ax2.fill_between(range(len(step_names_salary)), salaries, alpha=0.3, color='#2ca02c')ax2.set_xticks(range(len(step_names_salary)))ax2.set_xticklabels(step_names_salary, rotation=45, ha='right')ax2.set_ylabel('Salary ($)', fontsize=12)ax2.set_title('Salary Trajectory', fontsize=14, fontweight='bold')ax2.grid(alpha=0.3)for i, (name, salary) in enumerate(pathway.salary_trajectory):ax2.text(i, salary, f'${salary:,.0f}', ha='center', va='bottom', fontsize=9)plt.tight_layout()plt.show()else:print("Please run Section 9 first to generate career pathway.")


## 10. Learning RecommendationsGet personalized learning recommendations to fill skill gaps.

In [ ]:
def recommend_learning_resources(user: UserProfile, target_occupation: Occupation, gap_analysis: Dict) -> List[Dict]:"""Recommend learning resources based on skill gaps.Returns:List of recommended learning resources with priority, duration, cost, etc."""recommendations = []# Map skills to learning resources (in production, this would query a learning resource database)skill_to_resources = {'programming': [{'name': 'Python for Data Science', 'provider': 'Coursera', 'duration': '8 weeks', 'cost': 49, 'format': 'Online'},{'name': 'Advanced Python Programming', 'provider': 'edX', 'duration': '12 weeks', 'cost': 99, 'format': 'Online'}],'problem_solving': [{'name': 'Algorithm Design and Analysis', 'provider': 'MIT OpenCourseWare', 'duration': '16 weeks', 'cost': 0, 'format': 'Online'},],'software_design': [{'name': 'Software Architecture Patterns', 'provider': 'Udemy', 'duration': '6 weeks', 'cost': 19, 'format': 'Online'},],'testing': [{'name': 'Software Testing Fundamentals', 'provider': 'Pluralsight', 'duration': '4 weeks', 'cost': 29, 'format': 'Online'},],'documentation': [{'name': 'Technical Writing for Developers', 'provider': 'LinkedIn Learning', 'duration': '2 weeks', 'cost': 29, 'format': 'Online'},]}# Prioritize missing critical skillsfor skill in gap_analysis['missing_skills'][:10]: # Top 10 missingskill_key = skill['skill_id'].lower()resources = skill_to_resources.get(skill_key, [])if not resources:# Generic resource if specific not foundresources = [{'name': f'{skill["skill_name"]} Course','provider': 'Various','duration': '8 weeks','cost': 50,'format': 'Online'}]for resource in resources:recommendations.append({'skill': skill['skill_name'],'priority': skill['priority'],'required_importance': skill['required_importance'],'resource_name': resource['name'],'provider': resource['provider'],'duration': resource['duration'],'cost': resource['cost'],'format': resource['format'],'roi_estimate': (skill['required_importance'] * 10000) - resource['cost'] # Simple ROI estimate})# Add recommendations for weak skillsfor skill in gap_analysis['weak_skills'][:5]: # Top 5 weakskill_key = skill['skill_id'].lower()resources = skill_to_resources.get(skill_key, [])if resources:for resource in resources[:1]: # One resource per weak skillrecommendations.append({'skill': skill['skill_name'],'priority': 'improvement','required_importance': skill['required_importance'],'user_proficiency': skill['user_proficiency'],'gap': skill['gap'],'resource_name': resource['name'],'provider': resource['provider'],'duration': resource['duration'],'cost': resource['cost'],'format': resource['format'],'roi_estimate': (skill['gap'] * 5000) - resource['cost']})# Sort by priority and ROIpriority_order = {'critical': 3, 'important': 2, 'nice-to-have': 1, 'improvement': 1}recommendations.sort(key=lambda x: (priority_order.get(x['priority'], 0), x.get('roi_estimate', 0)), reverse=True)return recommendations# Generate learning recommendationsprint("="*80)print("LEARNING RECOMMENDATIONS")print("="*80)if 'gap_analysis' in globals() and 'target_occupation' in globals():recommendations = recommend_learning_resources(resume_user, target_occupation, gap_analysis)print(f"\nRecommended Learning Resources ({len(recommendations)} total):")print("="*80)for i, rec in enumerate(recommendations[:15], 1): # Top 15print(f"\n{i}. {rec['resource_name']}")print(f"Skill: {rec['skill']} (Priority: {rec['priority']})")print(f"Provider: {rec['provider']}")print(f"Duration: {rec['duration']} | Cost: ${rec['cost']} | Format: {rec['format']}")if 'roi_estimate' in rec:print(f"Estimated ROI: ${rec['roi_estimate']:,.0f}")print("\n" + "="*80)else:print("\nWARNING: Please run Section 7 first to generate gap analysis.")print("="*80)


### 10B: Learning Recommendations Visualization

In [ ]:
if 'recommendations' in globals() and len(recommendations) > 0:# Visualize learning recommendationsfig, axes = plt.subplots(1, 2, figsize=(16, 6))# Recommendations by prioritypriority_counts = {}for rec in recommendations:priority = rec['priority']priority_counts[priority] = priority_counts.get(priority, 0) + 1axes[0].bar(priority_counts.keys(), priority_counts.values(),color=['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4'])axes[0].set_ylabel('Number of Recommendations', fontsize=12)axes[0].set_title('Learning Recommendations by Priority', fontsize=14, fontweight='bold')axes[0].grid(axis='y', alpha=0.3)# Cost vs ROItop_recs = recommendations[:10]costs = [rec['cost'] for rec in top_recs]rois = [rec.get('roi_estimate', 0) for rec in top_recs]rec_names = [rec['resource_name'][:30] for rec in top_recs]scatter = axes[1].scatter(costs, rois, s=100, alpha=0.6, c=range(len(top_recs)), cmap='viridis')axes[1].set_xlabel('Cost ($)', fontsize=12)axes[1].set_ylabel('Estimated ROI ($)', fontsize=12)axes[1].set_title('Cost vs ROI: Top 10 Recommendations', fontsize=14, fontweight='bold')axes[1].grid(alpha=0.3)# Add labels for top recommendationsfor i, (cost, roi, name) in enumerate(zip(costs, rois, rec_names)):if i < 5: # Label top 5axes[1].annotate(name, (cost, roi), fontsize=8, alpha=0.7)plt.tight_layout()plt.show()# Summary statisticstotal_cost = sum(rec['cost'] for rec in recommendations)total_roi = sum(rec.get('roi_estimate', 0) for rec in recommendations)avg_duration = sum(int(rec['duration'].split()[0]) for rec in recommendations if rec['duration'].split()[0].isdigit()) / len(recommendations)print(f"\nSummary:")print(f"• Total Recommendations: {len(recommendations)}")print(f"• Total Investment: ${total_cost:,.0f}")print(f"• Total Estimated ROI: ${total_roi:,.0f}")print(f"• Average Duration: {avg_duration:.1f} weeks")else:print("Please run Section 10 first to generate learning recommendations.")


## 11. Integrated DashboardComplete dashboard with overview metrics, visualizations, and action items.

In [ ]:
def calculate_career_readiness_score(user: UserProfile, target_occupation: Occupation, gap_analysis: Dict) -> float:"""Calculate overall career readiness score (0-100)."""# Base score from occupation matchingif 'occupation_matches' in globals():for occ, score in occupation_matches:if occ.soc_code == target_occupation.soc_code:base_score = scorebreakelse:base_score = 50else:base_score = 50# Adjust for gap severitygap_penalty = min(30, gap_analysis['gap_severity'] * 2)readiness_score = max(0, base_score - gap_penalty)return readiness_score# Integrated Dashboardprint("="*80)print("SKILLFORGE INTEGRATED DASHBOARD")print("="*80)if 'resume_user' in globals():# Calculate readiness scoreif 'target_occupation' in globals() and 'gap_analysis' in globals():readiness_score = calculate_career_readiness_score(resume_user, target_occupation, gap_analysis)else:readiness_score = 0# Get top 3 occupation matchestop_3_occupations = []if 'occupation_matches' in globals():top_3_occupations = [(occ.title, score) for occ, score in occupation_matches[:3]]# Get critical skill gaps countcritical_gaps = 0if 'gap_analysis' in globals():critical_gaps = gap_analysis['critical_missing_count']# Get market salary rangesalary_range = Noneif 'market_data' in globals():if market_data.starting_wage and market_data.exit_wage:salary_range = (market_data.starting_wage, market_data.exit_wage)print(f"\nOVERVIEW METRICS")print("="*80)print(f"Career Readiness Score: {readiness_score:.1f}/100")print(f"Top 3 Occupation Matches: {len(top_3_occupations)}")print(f"Critical Skill Gaps: {critical_gaps}")if salary_range:print(f"Market Salary Range: ${salary_range[0]:,.0f} - ${salary_range[1]:,.0f}")print(f"\nTOP 3 OCCUPATION MATCHES")print("="*80)for i, (title, score) in enumerate(top_3_occupations, 1):print(f"{i}. {title} ({score:.1f}% match)")if 'gap_analysis' in globals():print(f"\nSKILL GAP SUMMARY")print("="*80)print(f"Missing Skills: {len(gap_analysis['missing_skills'])}")print(f"• Critical: {gap_analysis['critical_missing_count']}")print(f"• Important: {gap_analysis['important_missing_count']}")print(f"Weak Skills: {gap_analysis['weak_skills_count']}")print(f"Strong Skills: {len(gap_analysis['strong_skills'])}")if 'pathway' in globals():print(f"\nCAREER PATHWAY")print("="*80)print(f"Path: {pathway.current_occupation.title} → {pathway.target_occupation.title}")print(f"Total Estimated Time: {pathway.total_estimated_time} months")print(f"Number of Steps: {len(pathway.steps)}")if 'recommendations' in globals():print(f"\nLEARNING RECOMMENDATIONS")print("="*80)print(f"Total Recommendations: {len(recommendations)}")critical_recs = [r for r in recommendations if r['priority'] == 'critical']print(f"Critical Priority: {len(critical_recs)}")if recommendations:total_cost = sum(r['cost'] for r in recommendations)print(f"Total Investment: ${total_cost:,.0f}")print(f"\nACTION ITEMS")print("="*80)if 'gap_analysis' in globals() and gap_analysis['missing_skills']:print("Next Skills to Acquire:")for skill in gap_analysis['missing_skills'][:5]:print(f"• {skill['skill_name']} (Priority: {skill['priority']})")if 'recommendations' in globals() and recommendations:print("\nRecommended Courses:")for rec in recommendations[:3]:print(f"• {rec['resource_name']} ({rec['provider']})")if 'occupation_matches' in globals():print("\nTarget Occupations to Explore:")for occ, score in occupation_matches[:3]:print(f"• {occ.title} ({score:.1f}% match)")# Create comprehensive visualizationif 'occupation_matches' in globals() and 'gap_analysis' in globals():fig = plt.figure(figsize=(16, 10))gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)# 1. Readiness Score (gauge-like)ax1 = fig.add_subplot(gs[0, 0])ax1.text(0.5, 0.5, f'{readiness_score:.0f}', ha='center', va='center',fontsize=48, fontweight='bold', color='#2ca02c' if readiness_score >= 70 else '#ff7f0e' if readiness_score >= 50 else '#d62728')ax1.text(0.5, 0.3, 'Career Readiness', ha='center', va='center', fontsize=14)ax1.set_xlim(0, 1)ax1.set_ylim(0, 1)ax1.axis('off')# 2. Top Matchesax2 = fig.add_subplot(gs[0, 1])if top_3_occupations:titles = [t[:30] for t, _ in top_3_occupations]scores = [s for _, s in top_3_occupations]ax2.barh(titles, scores, color=sns.color_palette("viridis", len(titles)))ax2.set_xlabel('Match Score (%)')ax2.set_title('Top 3 Matches')ax2.set_xlim(0, 100)# 3. Gap Summaryax3 = fig.add_subplot(gs[0, 2])gap_categories = ['Critical\nMissing', 'Important\nMissing', 'Weak\nSkills']gap_counts = [gap_analysis['critical_missing_count'],gap_analysis['important_missing_count'],gap_analysis['weak_skills_count']]ax3.bar(gap_categories, gap_counts, color=['#d62728', '#ff7f0e', '#ffbb33'])ax3.set_ylabel('Count')ax3.set_title('Skill Gap Summary')# 4. Salary Trajectoryax4 = fig.add_subplot(gs[1, :2])if 'pathway' in globals() and pathway.salary_trajectory:step_names = [name for name, _ in pathway.salary_trajectory]salaries = [sal for _, sal in pathway.salary_trajectory]ax4.plot(range(len(step_names)), salaries, marker='o', linewidth=3, markersize=12, color='#2ca02c')ax4.fill_between(range(len(step_names)), salaries, alpha=0.3, color='#2ca02c')ax4.set_xticks(range(len(step_names)))ax4.set_xticklabels(step_names, rotation=45, ha='right')ax4.set_ylabel('Salary ($)', fontsize=12)ax4.set_title('Career Pathway Salary Trajectory', fontsize=14, fontweight='bold')ax4.grid(alpha=0.3)for i, (name, salary) in enumerate(pathway.salary_trajectory):ax4.text(i, salary, f'${salary:,.0f}', ha='center', va='bottom', fontsize=10)# 5. Learning Recommendations Priorityax5 = fig.add_subplot(gs[1, 2])if 'recommendations' in globals():priority_counts = {}for rec in recommendations:priority = rec['priority']priority_counts[priority] = priority_counts.get(priority, 0) + 1if priority_counts:ax5.pie(priority_counts.values(), labels=priority_counts.keys(), autopct='%1.0f',colors=['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4'])ax5.set_title('Learning Recommendations\nby Priority')plt.suptitle('SkillForge Integrated Dashboard', fontsize=16, fontweight='bold', y=0.98)plt.show()print("\n" + "="*80)print("Dashboard Complete!")print("="*80)else:print("\nWARNING: Please upload your resume in Section 3 first.")print("="*80)


## 12. Skill VisualizationVisualize the skills extracted from your resume by JAAT.

In [ ]:
if 'resume_user' in globals() and resume_user.skills:skill_names = list(resume_user.skills.keys())skill_proficiencies = list(resume_user.skills.values())fig, ax = plt.subplots(figsize=(12, 8))bars = ax.barh(skill_names, skill_proficiencies, color=sns.color_palette("viridis", len(skill_names)))ax.set_xlabel('Proficiency Level (1-5)', fontsize=12)ax.set_title('Skills Extracted from Resume by JAAT', fontsize=14, fontweight='bold')ax.set_xlim(0, 5)ax.grid(axis='x', alpha=0.3)for i, (name, prof) in enumerate(zip(skill_names, skill_proficiencies)):ax.text(prof + 0.1, i, f"{prof}/5", va='center', fontsize=10)plt.tight_layout()plt.show()print(f"\nTotal Skills: {len(skill_names)}")print(f"Average Proficiency: {sum(skill_proficiencies) / len(skill_proficiencies):.1f}/5")print(f"Highest Proficiency: {max(skill_proficiencies)}/5")else:print("WARNING: Please upload your resume in Section 3 first.")


## 13. Feature Vector AnalysisAnalyze the JAAT feature vectors extracted from your resume.

In [ ]:
if 'resume_user' in globals() and resume_user.jaat_feature_weights:features = list(resume_user.jaat_feature_weights.keys())weights = [w if isinstance(w, (int, float)) else 0 for w in resume_user.jaat_feature_weights.values()]sorted_data = sorted(zip(features, weights), key=lambda x: x[1], reverse=True)top_features = [f[0] for f in sorted_data[:15]]top_weights = [f[1] for f in sorted_data[:15]]fig, ax = plt.subplots(figsize=(14, 8))bars = ax.barh(top_features, top_weights, color=sns.color_palette("viridis", len(top_features)))ax.set_xlabel('Feature Weight', fontsize=12)ax.set_title('Top JAAT Feature Vectors (from NLx Corpus)', fontsize=14, fontweight='bold')ax.grid(axis='x', alpha=0.3)for i, (feat, weight) in enumerate(zip(top_features, top_weights)):ax.text(weight + 0.01, i, f"{weight:.3f}", va='center', fontsize=9)plt.tight_layout()plt.show()print(f"\nTotal Feature Vectors: {len(features)}")print(f"Average Weight: {sum(weights) / len(weights):.3f}")print(f"Max Weight: {max(weights):.3f}")print("\nSource: JAAT Toolkit & NLx Corpus")else:print("WARNING: Please upload your resume in Section 3 first.")


## SummaryThis notebook demonstrates the complete SkillForge capabilities:### Implemented Features1. **Skill-to-Occupation Matching** - Match user profile to O*NET occupations based on skills, experience, and education2. **Skill Gap Analysis** - Identify missing and weak skills for target occupations with priority levels3. **Career Pathway Planning** - Generate step-by-step career progression paths with salary trajectories4. **Market Data Integration** - Real market salaries from DOL Apprenticeship Data API and demand trends analysis### Data Sources Used- **O*NET Database**: Occupations, skills, knowledge requirements- **DOL Apprenticeship Data API**: Real market wages by occupation/state- **O*NET-SOC Code Labels**: Occupation mapping- **JAAT Toolkit**: Skill extraction from resumes using NLx corpus### Key Functions Implemented- `load_onet_soc_labels()`: Load O*NET occupation labels- `load_dol_apprenticeship_data()`: Fetch DOL market data with caching- `match_user_to_occupations()`: Match user to occupations with scoring- `calculate_skill_gap()`: Analyze skill gaps with priority levels- `generate_career_pathway()`: Generate career pathways with transitional occupations- `recommend_learning_resources()`: Personalized learning recommendations- `get_market_salary()`: Query real market salaries from DOL data### Next Steps for Production1. **Integrate NLx Aggregated Datasets**: Enhanced market analysis with monthly trend data (2015-2025)2. **Enhanced Learning Resource Database**: Connect to real course/certification APIs3. **Real-time Job Posting Analysis**: Integrate with job board APIs4. **Advanced Pathway Optimization**: ML-based pathway recommendations5. **User Progress Tracking**: Track skill acquisition over time### Architecture- **JAAT Foundation**: Provides NLP capabilities for skill extraction- **SkillForge Enhancement**: Adds value-added features (matching, pathways, recommendations)- **Real Data Integration**: Uses DOL and O*NET data, not mock data- **Modular Design**: Each section can be run independently---**All features use real data (DOL, O*NET) and JAAT extraction - no mock data!**